In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
main_file = pd.ExcelFile("/Users/timothypark/Documents/portfolios/timpark99.github.io/supply chain analysis/supply chain original.xlsx")
df_dict = {sheet_name: main_file.parse(sheet_name) for sheet_name in main_file.sheet_names}
df_dict.keys()

dict_keys(['OrderList', 'FreightRates', 'WhCosts', 'WhCapacities', 'ProductsPerPlant', 'VmiCustomers', 'PlantPorts'])

In [5]:
for df_name, df in df_dict.items():
    print(f"{df_name} - Shape: {df.shape}")

    duplicate_count = df.duplicated().sum()
    missing_values_count = df.isnull().sum().sum()

    if duplicate_count > 0 or missing_values_count > 0:
        print(f" >>>{df_name} - Duplicates: {duplicate_count}; Missing Values: {missing_values_count}")
    else:
        print(f" >>>{df_name} has no duplicates or missing values.")

OrderList - Shape: (9215, 14)
 >>>OrderList has no duplicates or missing values.
FreightRates - Shape: (1540, 11)
 >>>FreightRates - Duplicates: 3; Missing Values: 0
WhCosts - Shape: (19, 2)
 >>>WhCosts has no duplicates or missing values.
WhCapacities - Shape: (19, 2)
 >>>WhCapacities has no duplicates or missing values.
ProductsPerPlant - Shape: (2036, 2)
 >>>ProductsPerPlant has no duplicates or missing values.
VmiCustomers - Shape: (14, 2)
 >>>VmiCustomers has no duplicates or missing values.
PlantPorts - Shape: (22, 2)
 >>>PlantPorts has no duplicates or missing values.


In [9]:
FreightRates = df_dict['FreightRates'].drop_duplicates()
df_dict['FreightRates'] = FreightRates

for df_name, df in df_dict.items():
    df.columns = [col.strip().replace(' ', '_').replace('/', '_').upper() for col in df.columns]

print(df_dict['OrderList'].columns)


Index(['ORDER_ID', 'ORDER_DATE', 'ORIGIN_PORT', 'CARRIER', 'TPT',
       'SERVICE_LEVEL', 'SHIP_AHEAD_DAY_COUNT', 'SHIP_LATE_DAY_COUNT',
       'CUSTOMER', 'PRODUCT_ID', 'PLANT_CODE', 'DESTINATION_PORT',
       'UNIT_QUANTITY', 'WEIGHT'],
      dtype='object')


In [ ]:
orderList = df_dict['OrderList']
orderList = orderList.merge(df_dict['FreightRates'], left_on = ['CARRIER', 'ORIGIN_PORT', 'DESTINATION_PORT'], right_on=['CARRIER', 'ORIG_PORT_CD', 'DEST_PORT_CD'], how='left')
orderList = orderList.merge(df_dict['WhCosts'], left_on='PLANT_CODE', right_on='WH', how='left')
orderList['COST'] = (orderList['UNIT_QUANTITY']*orderList['RATE']) + (orderList['UNIT_QUANTITY']*orderList['COST_UNIT'])
orderList = orderList.dropna()
orderList['ORDER_DATE'] = pd.to_datetime(orderList['ORDER_DATE'])
productsPerPlant = df_dict['ProductsPerPlant']
productsPerPlant = productsPerPlant.dropna()

df_dict['OrderList'] = orderList
df_dict['ProductsPerPlant'] = productsPerPlant

